In [1]:
# dependencies
import datetime
import numpy as np
import pandas as pd

In [2]:
# support methods

In [3]:
# __main__
mp_f = "../../clean/output/mp.parquet"
extras_f = "../../join/output/extras.parquet"
mp = pd.read_parquet(mp_f, engine="fastparquet")
extras = pd.read_parquet(extras_f)

In [4]:
core_summary = ['race', 'sex', 'age_group', 'ttc_group', 'previously_missing',
                'current_iucr', 'primary', 'description',
                'occurrence_date', 'closed_date', 'days_to_arrive',
                'status_date', 'status_temp', 'status_mp', 'last_update', 'updated']

### overview

In [5]:
mp.memory_usage()

Index                  128
rd_no              2840424
value_conflict      355053
race               2840424
sex                2840424
                    ...   
estimated_age      2840424
age_is_estimate     355053
age_w_unit         2840424
no_age_data        2840424
age_group          2840424
Length: 71, dtype: int64

In [6]:
mp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 355053 entries, 0 to 355052
Data columns (total 70 columns):
 #   Column                      Non-Null Count   Dtype          
---  ------                      --------------   -----          
 0   rd_no                       355053 non-null  object         
 1   value_conflict              355053 non-null  bool           
 2   race                        352014 non-null  object         
 3   sex                         352266 non-null  object         
 4   given_age_group             149513 non-null  object         
 5   beat_assigned               355050 non-null  object         
 6   no__times_missing           57256 non-null   object         
 7   date_last_missing           32447 non-null   datetime64[ns] 
 8   length_missing              45381 non-null   object         
 9   how_located                 47306 non-null   object         
 10  place_of_birth              136802 non-null  object         
 11  relation_to_missing       

In [7]:
mp.sample(5)

,rd_no,value_conflict,race,sex,given_age_group,beat_assigned,no__times_missing,date_last_missing,length_missing,how_located,...,DTC2_group,DTC3_group,original_year_of_birth,original_age,year_of_birth_modified,estimated_age,age_is_estimate,age_w_unit,no_age_data,age_group
86321,HN415275,True,BLACK,F,None,2132,None,NaT,None,None,...,MISSING,MISSING,1991,16.0,False,16.0,False,16 (Y),0,youth (10-20)
95511,F780938,True,BLACK,M,None,1854,None,NaT,None,None,...,MISSING,MISSING,1955,45.0,False,45.0,False,45 (Y),0,adult (21-59)
137740,HJ498989,True,BLACK,M,None,0952,None,NaT,None,None,...,MISSING,MISSING,1987,16.0,False,16.0,False,16 (Y),0,youth (10-20)
193825,HT205641,True,BLACK,M,None,0702,None,NaT,None,None,...,Over a year,MISSING,1992,18.0,False,19.0,False,18 (Y),0,youth (10-20)
210347,HL574848,True,BLACK,M,None,2454,None,NaT,None,None,...,MISSING,MISSING,1963,42.0,False,42.0,False,42 (Y),0,adult (21-59)


## `sex`

### unkown 

In [8]:
na_rdnos = mp.loc[mp.sex.isnull(), 'rd_no'].unique()
na_rdnos.shape[0]

2787

In [9]:
any([rdno in extras.rd_no for rdno in na_rdnos])

False

In [10]:
# 8 sets of records with unique `rd`s share case level details
# with 
# - status, date_occurred, date_officer_arrived, 
#   closed_date, last_update
# 
(mp.loc[mp.sex.isna(),
       ['status', 'date_occurred', 'date_officer_arrived', 
        'closed_date', 'last_update']].value_counts() > 1).sum()

8

In [11]:
# sample 1
mp.loc[(mp.date_occurred == '2002-06-24 12:00:00') & 
       (mp.date_officer_arrived == '2002-06-24 19:25:00') & 
       (mp.closed_date == '2002-07-03') & 
       (mp.last_update == '2002-07-03 18:28:00')]

,rd_no,value_conflict,race,sex,given_age_group,beat_assigned,no__times_missing,date_last_missing,length_missing,how_located,...,DTC2_group,DTC3_group,original_year_of_birth,original_age,year_of_birth_modified,estimated_age,age_is_estimate,age_w_unit,no_age_data,age_group
596,HH464772,True,None,None,10-16,1754,None,NaT,None,None,...,MISSING,MISSING,1988,13.0,False,14.0,False,13 (Y),0,youth (10-20)
1515,HH464771,True,None,None,10-16,1754,None,NaT,None,None,...,MISSING,MISSING,1987,14.0,False,15.0,False,14 (Y),0,youth (10-20)


In [12]:
# sample 2
mp.loc[(mp.date_occurred == '2007-11-02 11:55:00') & 
       (mp.date_officer_arrived == '2007-11-06 11:15:00') & 
       (mp.closed_date == '2007-11-21') & 
       (mp.last_update == '2007-11-21 17:02:00')]

,rd_no,value_conflict,race,sex,given_age_group,beat_assigned,no__times_missing,date_last_missing,length_missing,how_located,...,DTC2_group,DTC3_group,original_year_of_birth,original_age,year_of_birth_modified,estimated_age,age_is_estimate,age_w_unit,no_age_data,age_group
275861,HN692025,True,None,None,None,1942,None,NaT,None,None,...,MISSING,Over a year,None,NaN,False,NaN,False,None,1,None
304016,HN692026,True,None,None,None,1942,None,NaT,None,None,...,MISSING,Over a year,None,NaN,False,NaN,False,None,1,None


In [13]:
# sample 3
mp.loc[(mp.date_occurred == '2002-10-24 17:30:00') & 
       (mp.date_officer_arrived == '2002-10-24 20:10:00') & 
       (mp.closed_date == '2002-10-29') & 
       (mp.last_update == '2002-10-29 17:47:00')]

,rd_no,value_conflict,race,sex,given_age_group,beat_assigned,no__times_missing,date_last_missing,length_missing,how_located,...,DTC2_group,DTC3_group,original_year_of_birth,original_age,year_of_birth_modified,estimated_age,age_is_estimate,age_w_unit,no_age_data,age_group
21845,HH736151,True,None,None,10-16,1722,None,NaT,None,None,...,MISSING,MISSING,1990,12.0,False,12.0,False,12 (Y),0,youth (10-20)
48557,HH736148,True,None,None,10-16,1722,2.0,NaT,1 DAY,WITH FAMILY,...,MISSING,MISSING,1990,12.0,False,12.0,False,12 (Y),0,youth (10-20)


## `sex` && `previously_missing`

In [14]:
sorted(core_summary)

['age_group',
 'closed_date',
 'current_iucr',
 'days_to_arrive',
 'description',
 'last_update',
 'occurrence_date',
 'previously_missing',
 'primary',
 'race',
 'sex',
 'status_date',
 'status_mp',
 'status_temp',
 'ttc_group',
 'updated']

In [15]:
mp.previously_missing = mp.previously_missing.fillna('Unknown')
mp.loc[:, ['race' , 'sex', 'age_group', 
           'ttc_group', 'previously_missing']
      ].groupby(['previously_missing', 'sex']).describe()

race                      age_group         \
                         count unique    top   freq     count unique   
previously_missing sex                                                 
N                  F    137494      6  BLACK  98570    137509      4   
                   M    139756      6  BLACK  98825    139767      4   
Unknown            F      3066      6  BLACK   2183      3069      4   
                   M      2701      6  BLACK   1714      2703      4   
Y                  F     38556      6  BLACK  29204     38575      4   
                   M     30386      6  BLACK  23539     30399      4   

                                              ttc_group                        \
                                  top    freq     count unique            top   
previously_missing sex                                                          
N                  F    youth (10-20)  107606    137619     10  under 14 days   
                   M    youth (10-20)   88967    139876     10  under 14 days   
Unknown            F    youth (10-20)    2152      3071      1        MISSING   
                   M    youth (10-20)    1217      2707      1        MISSING   
Y                  F    youth (10-20)   35177     38584     10  under 14 days   
                   M    youth (10-20)   24877     30409     10  under 14 days   

                               
                         freq  
previously_missing sex         
N                  F    37011  
                   M    36101  
Unknown            F     3071  
                   M     2707  
Y                  F    12868  
                   M    10589

In [16]:
mp.loc[:, ['race' , 'sex', 'age_group', 
           'ttc_group', 'previously_missing']
      ].groupby(['previously_missing', 'race']).describe()

sex                    \
                                             count unique top   freq   
previously_missing race                                                
N                  AMER IND/ALASKAN NATIVE     230      2   M    133   
                   ASIAN/PACIFIC ISLANDER     2645      2   M   1368   
                   BLACK                    197395      2   M  98825   
                   BLACK HISPANIC             1006      2   M    512   
                   WHITE                     31715      2   M  18315   
                   WHITE HISPANIC            44259      2   F  23656   
Unknown            AMER IND/ALASKAN NATIVE       6      2   M      4   
                   ASIAN/PACIFIC ISLANDER       88      2   F     47   
                   BLACK                      3897      2   F   2183   
                   BLACK HISPANIC               43      2   F     24   
                   WHITE                       743      2   M    425   
                   WHITE HISPANIC              990      2   M    498   
Y                  AMER IND/ALASKAN NATIVE      42      2   F     26   
                   ASIAN/PACIFIC ISLANDER      349      2   M    182   
                   BLACK                     52743      2   F  29204   
                   BLACK HISPANIC              318      2   F    178   
                   WHITE                      6278      2   F   3428   
                   WHITE HISPANIC             9212      2   F   5553   

                                           age_group                        \
                                               count unique            top   
previously_missing race                                                      
N                  AMER IND/ALASKAN NATIVE       230      4  youth (10-20)   
                   ASIAN/PACIFIC ISLANDER       2638      4  adult (21-59)   
                   BLACK                      197286      4  youth (10-20)   
                   BLACK HISPANIC               1002      4  youth (10-20)   
                   WHITE                       31695      4  youth (10-20)   
                   WHITE HISPANIC              44239      4  youth (10-20)   
Unknown            AMER IND/ALASKAN NATIVE         6      3  youth (10-20)   
                   ASIAN/PACIFIC ISLANDER         87      3  adult (21-59)   
                   BLACK                        3897      4  youth (10-20)   
                   BLACK HISPANIC                 43      4  youth (10-20)   
                   WHITE                         742      4  adult (21-59)   
                   WHITE HISPANIC                988      4  youth (10-20)   
Y                  AMER IND/ALASKAN NATIVE        42      3  youth (10-20)   
                   ASIAN/PACIFIC ISLANDER        349      4  youth (10-20)   
                   BLACK                       52734      4  youth (10-20)   
                   BLACK HISPANIC                318      4  youth (10-20)   
                   WHITE                        6276      4  youth (10-20)   
                   WHITE HISPANIC               9212      4  youth (10-20)   

                                                   ttc_group         \
                                              freq     count unique   
previously_missing race                                               
N                  AMER IND/ALASKAN NATIVE     108       230      9   
                   ASIAN/PACIFIC ISLANDER     1182      2645     10   
                   BLACK                    146640    197420     10   
                   BLACK HISPANIC              719      1006      9   
                   WHITE                     14748     31729     10   
                   WHITE HISPANIC            33286     44265     10   
Unknown            AMER IND/ALASKAN NATIVE       3         6      1   
                   ASIAN/PACIFIC ISLANDER       52        88      1   
                   BLACK                      2455      3899      1   
                   BLACK HISPANIC          

In [17]:
a = mp.loc[(mp.race == 'BLACK') & 
           (mp.sex == 'F') & 
           (mp.age_group.isin(['tender age (0-9)','youth (10-20)'])) & 
           (mp.previously_missing == 'N')].record_id.count()
b = mp.loc[(mp.race == 'BLACK') & 
           (mp.sex == 'M') & 
           (mp.age_group.isin(['tender age (0-9)','youth (10-20)'])) & 
           (mp.previously_missing == 'N')].record_id.count()
n = mp.loc[mp.previously_missing == 'N'].record_id.count()
print("===========================")
print('previously_missing:\t"N"')
print("===========================")
print('black minor, female:\t', round(a/n, 2))
print('black minor, male:\t', round(b/n, 2))
print('either:\t\t\t', round((a+b)/n, 2))

previously_missing:	"N"
black minor, female:	 0.29
black minor, male:	 0.25
either:			 0.55


In [18]:
a = mp.loc[(mp.race == 'BLACK') & 
           (mp.sex == 'F') & 
           (mp.age_group.isin(['tender age (0-9)','youth (10-20)'])) & 
           (mp.previously_missing == 'Y')].record_id.count()
b = mp.loc[(mp.race == 'BLACK') & 
           (mp.sex == 'M') & 
           (mp.age_group.isin(['tender age (0-9)','youth (10-20)'])) & 
           (mp.previously_missing == 'Y')].record_id.count()
n = mp.loc[mp.previously_missing == 'Y'].record_id.count()
print("===========================")
print('previously_missing:\t"Y"')
print("===========================")
print('black minor, female:\t', round(a/n, 2))
print('black minor, male:\t', round(b/n, 2))
print('either:\t\t\t', round((a+b)/n, 2))

previously_missing:	"Y"
black minor, female:	 0.39
black minor, male:	 0.3
either:			 0.69
